# Proyecto final de aplicación de machine learning

**Learning Code**

## Introdución

La estimación de los precios de la vivienda es esencial tanto para los propietarios como para los inversores, ya que ambos necesitan entender el valor de este activo inmobiliario. 
Para muchas personas, comprar una propiedad es una de las decisiones y compra más importantes en la vida. Además de la asequibilidad de una vivienda, otros factores, 
como la conveniencia del lugar y las perspectivas de inversión a largo plazo, también afectan
el proceso de toma de decisiones.
El mercado inmobiliario está expuesto a muchas fluctuaciones en los precios debido a las correlaciones existentes con muchas variables, algunas de las cuales no se pueden controlar
o incluso pueden ser desconocidas. Los precios de las viviendas pueden aumentar rápidamente (o en algunos casos, también bajan muy rápido).
Algunas aplicaciones para un banco son:
- Originación: Establecer el valor comercial del inmueble a financiar utilizado para la aprobación final.
- Retanqueo: Actualizar el valor comercial de garantía ya existente para aprobación de nuevos cupos de crédito.
- Monitoreo portafolio garantías: Valoración del portafolio de las garantías para cumplimiento normativo y para el análisis de riesgos del colateral.
- Normalización de cartera: Evaluar los préstamos existentes, evaluar los acuerdos de refinanciamiento Y daciones en pago.

## Objetivo:

El objetivo de este proyecto es que el estudiante aplique los temas vistos en el BootCamp para resolver un problema del mundo real, con datos reales. Lea cuidadosamente las instrucciones contenidas en el archivo "Enunciado_dataton_banco". Allí están los detalles iniciales de la prueba. Para este curso, se evaluará adicionalmente el desarrollo de los puntos de este notebook. 

Al final, usted debe entregar una copia de este notebook junto con unas conclusiones finales que usted debe sacar y que deben estar contenidas al final del documento.

Cómo es usual, para empezar, debemos importar todas las librerías que vamos a necesitar. Asegurese de contar con las librerias, hasta que no encuentre ningún error al correr la siguiente celda:

In [205]:
# import the necessary packages

import pandas as pd
import numpy as np
import argparse
import glob
from pandas import DataFrame
import sklearn
from sklearn import tree
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_rows', 500)


# Cargar los datos que usaremos

Usaremos los archivos entrenamiento_precios_vivienda.csv  y prueba_precios_vivienda.csv. Note que los datos del archivo prueba_precios_vivienda.csv no contienen la columna de los precios de la vivienda. La idea de este archivo, es que usted complete dicha columna con los predicciones resultantes de su modelo, y mediante un proceso de validación externo, Learning Code calcula el desempeño de este. Esta es una práctica muy común en pruebas de este tipo.

In [206]:
from content.utils.data_processing import loadTrainData,loadTestData

dataTrain = loadTrainData()
dataTest = loadTestData()

# New Section

# New Section

In [207]:
# Asignar el campo id como índice del dataframe
dataTrain['id'] = pd.to_numeric(dataTrain['id'], errors='coerce')
dataTest['id'] = pd.to_numeric(dataTest['id'], errors='coerce')

dataTrain = dataTrain.set_index('id')
dataTest =  dataTest.set_index('id')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

dataTrain.head()

,fecha_aprobaciÃ³n,objeto,motivo,proposito,tipo_avaluo,tipo_credito,tipo_subsidio,departamento_inmueble,municipio_inmueble,barrio,sector,direccion_inmueble_informe,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,telefono_en_el_sector,vias_pavimentadas,sardineles_en_las_vias,andenes_en_las_vias,estrato,barrio_legal,topografia_sector,condiciones_salubridad,transporte,demanda_interes,paradero,alumbrado,arborizacion,alamedas,ciclo_rutas,nivel_equipamiento_comercial,descripcion_general_sector,perspectivas_de_valorizacion,actualidad_edificadora,comportamiento_oferta_demanda,alcantarillado_en_el_predio,acueducto_en_el_predio,gas_en_el_predio,energia_en_el_predio,telefono_en_el_predio,tipo_inmueble,descripcion_tipo_inmueble,uso_actual,descripcion_uso_inmueble,clase_inmueble,descripcion_clase_inmueble,ocupante,sometido_a_propiedad_horizontal,altura_permitida,observaciones_altura_permitida,aislamiento_posterior,observaciones_aislamiento_posterior,aislamiento_lateral,observaciones_aislamiento_lateral,antejardin,observaciones_antejardin,indice_ocupacion,observaciones_indice_ocupacion,indice_construccion,observaciones_indice_construccion,predio_subdividido_fisicamente,unidades,contadores_agua,contadores_luz,accesorios,area_valorada,observaciones_generales_inmueble,condicion_ph,numero_piso,numero_de_edificios,area_actividad,uso_principal_ph,rph,porteria,citofono,bicicletero,piscina,tanque_de_agua,club_house,garaje_visitantes,teatrino,sauna,vigilancia_privada,tipo_vigilancia,administracion,vetustez,pisos_bodega,estructura,ajustes_sismoresistentes,cubierta,fachada,tipo_fachada,estructura_reforzada,danos_previos,material_de_construccion,detalle_material,iluminacion,ventilacion,irregularidad_planta,irregularidad_altura,observaciones_estructura,habitaciones,estar_habitacion,cuarto_servicio,closet,sala,comedor,bano_privado,bano_social,bano_servicio,cocina,estudio,balcon,terraza,patio_interior,jardin,zona_de_ropas,zona_verde_privada,local,oficina,bodega,observaciones_dependencias,estado_acabados_pisos,calidad_acabados_pisos,estado_acabados_muros,calidad_acabados_muros,estado_acabados_techos,calidad_acabados_techos,estado_acabados_madera,calidad_acabados_madera,estado_acabados_metal,calidad_acabados_metal,estado_acabados_banos,calidad_acabados_banos,estado_acabados_cocina,calidad_acabados_cocina,tipo_garaje,numero_total_de_garajes,total_cupos_parquedaro,numero_garaje_1,matricula_garaje_1,garaje_cubierto_1,garaje_doble_1,garaje_paralelo_1,garaje_servidumbre_1,numero_garaje_2,matricula_garaje_2,garaje_cubierto_2,garaje_doble_2,garaje_paralelo_2,garaje_servidumbre_2,numero_garaje_3,matricula_garaje_3,garaje_cubierto_3,garaje_doble_3,garaje_paralelo_3,garaje_servidumbre_3,numero_garaje_4,matricula_garaje_4,garaje_cubierto_4,garaje_doble_4,garaje_paralelo_4,garaje_servidumbre_4,numero_garaje_5,matricula_garaje_5,garaje_cubierto_5,garaje_doble_5,garaje_paralelo_5,garaje_servidumbre_5,tipo_deposito,numero_total_depositos,numero_deposito_1,matricula_inmobiliaria_deposito_1,numero_deposito_2,matricula_inmobiliaria_deposito_2,numero_deposito_3,matricula_inmobiliaria_deposito_3,numero_deposito_4,matricula_inmobiliaria_deposito_4,numero_deposito_5,matricula_inmobiliaria_deposito_5,observaciones_generales_construccion,metodo_valuacion_1,concepto_del_metodo_1,metodo_valuacion_2,concepto_del_metodo_2,metodo_valuacion_3,concepto_del_metodo_3,metodo_valuacion_4,concepto_del_metodo_4,metodo_valuacion_5,concepto_del_metodo_5,metodo_valuacion_6,concepto_del_metodo_6,metodo_valuacion_7,concepto_del_metodo_7,metodo_valuacion_8,concepto_del_metodo_8,metodo_valuacion_9,concepto_del_metodo_9,area_privada,valor_area_privada,area_garaje,valor_area_garaje,area_deposito,valor_area_deposito,area_terreno,valor_area_terreno,area_construccion,valor_area_construccion,area_otros,valor_area_otros,area_libre,valor_area_libre,valor_total_avaluo,valor_uvr,valor_avaluo_en_uvr,Longitud,Latitud
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [208]:
dataTrain.describe()

,fecha_aprobaciÃ³n,objeto,motivo,proposito,tipo_avaluo,tipo_credito,tipo_subsidio,departamento_inmueble,municipio_inmueble,barrio,sector,direccion_inmueble_informe,alcantarillado_en_el_sector,acueducto_en_el_sector,gas_en_el_sector,energia_en_el_sector,telefono_en_el_sector,vias_pavimentadas,sardineles_en_las_vias,andenes_en_las_vias,estrato,barrio_legal,topografia_sector,condiciones_salubridad,transporte,demanda_interes,paradero,alumbrado,arborizacion,alamedas,ciclo_rutas,nivel_equipamiento_comercial,descripcion_general_sector,perspectivas_de_valorizacion,actualidad_edificadora,comportamiento_oferta_demanda,alcantarillado_en_el_predio,acueducto_en_el_predio,gas_en_el_predio,energia_en_el_predio,telefono_en_el_predio,tipo_inmueble,descripcion_tipo_inmueble,uso_actual,descripcion_uso_inmueble,clase_inmueble,descripcion_clase_inmueble,ocupante,sometido_a_propiedad_horizontal,altura_permitida,observaciones_altura_permitida,aislamiento_posterior,observaciones_aislamiento_posterior,aislamiento_lateral,observaciones_aislamiento_lateral,antejardin,observaciones_antejardin,indice_ocupacion,observaciones_indice_ocupacion,indice_construccion,observaciones_indice_construccion,predio_subdividido_fisicamente,unidades,contadores_agua,contadores_luz,accesorios,area_valorada,observaciones_generales_inmueble,condicion_ph,numero_piso,numero_de_edificios,area_actividad,uso_principal_ph,rph,porteria,citofono,bicicletero,piscina,tanque_de_agua,club_house,garaje_visitantes,teatrino,sauna,vigilancia_privada,tipo_vigilancia,administracion,vetustez,pisos_bodega,estructura,ajustes_sismoresistentes,cubierta,fachada,tipo_fachada,estructura_reforzada,danos_previos,material_de_construccion,detalle_material,iluminacion,ventilacion,irregularidad_planta,irregularidad_altura,observaciones_estructura,habitaciones,estar_habitacion,cuarto_servicio,closet,sala,comedor,bano_privado,bano_social,bano_servicio,cocina,estudio,balcon,terraza,patio_interior,jardin,zona_de_ropas,zona_verde_privada,local,oficina,bodega,observaciones_dependencias,estado_acabados_pisos,calidad_acabados_pisos,estado_acabados_muros,calidad_acabados_muros,estado_acabados_techos,calidad_acabados_techos,estado_acabados_madera,calidad_acabados_madera,estado_acabados_metal,calidad_acabados_metal,estado_acabados_banos,calidad_acabados_banos,estado_acabados_cocina,calidad_acabados_cocina,tipo_garaje,numero_total_de_garajes,total_cupos_parquedaro,numero_garaje_1,matricula_garaje_1,garaje_cubierto_1,garaje_doble_1,garaje_paralelo_1,garaje_servidumbre_1,numero_garaje_2,matricula_garaje_2,garaje_cubierto_2,garaje_doble_2,garaje_paralelo_2,garaje_servidumbre_2,numero_garaje_3,matricula_garaje_3,garaje_cubierto_3,garaje_doble_3,garaje_paralelo_3,garaje_servidumbre_3,numero_garaje_4,matricula_garaje_4,garaje_cubierto_4,garaje_doble_4,garaje_paralelo_4,garaje_servidumbre_4,numero_garaje_5,matricula_garaje_5,garaje_cubierto_5,garaje_doble_5,garaje_paralelo_5,garaje_servidumbre_5,tipo_deposito,numero_total_depositos,numero_deposito_1,matricula_inmobiliaria_deposito_1,numero_deposito_2,matricula_inmobiliaria_deposito_2,numero_deposito_3,matricula_inmobiliaria_deposito_3,numero_deposito_4,matricula_inmobiliaria_deposito_4,numero_deposito_5,matricula_inmobiliaria_deposito_5,observaciones_generales_construccion,metodo_valuacion_1,concepto_del_metodo_1,metodo_valuacion_2,concepto_del_metodo_2,metodo_valuacion_3,concepto_del_metodo_3,metodo_valuacion_4,concepto_del_metodo_4,metodo_valuacion_5,concepto_del_metodo_5,metodo_valuacion_6,concepto_del_metodo_6,metodo_valuacion_7,concepto_del_metodo_7,metodo_valuacion_8,concepto_del_metodo_8,metodo_valuacion_9,concepto_del_metodo_9,area_privada,valor_area_privada,area_garaje,valor_area_garaje,area_deposito,valor_area_deposito,area_terreno,valor_area_terreno,area_construccion,valor_area_construccion,area_otros,valor_area_otros,area_libre,valor_area_libre,valor_total_avaluo,valor_uvr,valor_avaluo_en_uvr,Longitud,Latitud
count,4061,9682,9682,9682,9682,9682,1149,9682,9682,9678

In [209]:
dataTrain.isnull().sum()

fecha_aprobaciÃ³n                       5621
objeto                                     0
motivo                                     0
proposito                                  0
tipo_avaluo                                0
tipo_credito                               0
tipo_subsidio                           8533
departamento_inmueble                      0
municipio_inmueble                         0
barrio                                     4
sector                                     1
direccion_inmueble_informe                 0
alcantarillado_en_el_sector                0
acueducto_en_el_sector                     0
gas_en_el_sector                           0
energia_en_el_sector                       0
telefono_en_el_sector                      0
vias_pavimentadas                          0
sardineles_en_las_vias                     0
andenes_en_las_vias                        0
estrato                                    0
barrio_legal                               0
topografia

Pocos datos faltantes, pero los pocos que no estan pueden rellenarse facilmente. El tipo de subsidio tiene demasiados datos faltantes.

In [210]:
dataTest.isnull().sum()

fecha_aprobaciÃ³n                       1892
objeto                                     0
motivo                                     0
proposito                                  0
tipo_avaluo                                0
tipo_credito                               0
tipo_subsidio                           2831
departamento_inmueble                      0
municipio_inmueble                         0
barrio                                     2
sector                                     0
direccion_inmueble_informe                 0
alcantarillado_en_el_sector                0
acueducto_en_el_sector                     0
gas_en_el_sector                           0
energia_en_el_sector                       0
telefono_en_el_sector                      0
vias_pavimentadas                          0
sardineles_en_las_vias                     0
andenes_en_las_vias                        0
estrato                                    0
barrio_legal                               0
topografia

Aparte de los datos faltantes en el valor, se nota que el tipo de subsidio tiene demasiados datos faltantes. En los demas pueden rellenarse los datos.

Clean dataset

Una columna que tiene demasiados datos faltantes en ambos dataframes es el de fecha de aprobación, por lo que se removera. Tambien podemos alegar que la fecha no deberia ser relevante, por lo menos en comparacion a otros parametros.

Tipo de subsidio tambien vemos que presenta muchos datos nulos, se eliminira.

Las columnas que consideramos que tienen poca informacion relevante y deben borrarse son:

[
  'fecha_aprobación',
  'tipo_subsidio',
  'numero_garaje_1',
  'matricula_garaje_1',
  'numero_garaje_2',
  'matricula_garaje_2',
  'numero_garaje_3',
  'matricula_garaje_3',
  'numero_garaje_4',
  'matricula_garaje_4',
  'numero_garaje_5',
  'matricula_garaje_5',
  'numero_deposito_1',
  'matricula_inmobiliaria_deposito_1',
  'numero_deposito_2',
  'matricula_inmobiliaria_deposito_2',
  'numero_deposito_3',
  'matricula_inmobiliaria_deposito_3',
  'numero_deposito_4',
  'matricula_inmobiliaria_deposito_4',
  'numero_deposito_5',
  'matricula_inmobiliaria_deposito_5',
  'metodo_valuacion_1',
  'concepto_del_metodo_1',
  'metodo_valuacion_2',
  'concepto_del_metodo_2',
  'metodo_valuacion_3',
  'concepto_del_metodo_3',
  'metodo_valuacion_4',
  'concepto_del_metodo_4',
  'metodo_valuacion_5',
  'concepto_del_metodo_5',
  'metodo_valuacion_6',
  'concepto_del_metodo_6',
  'metodo_valuacion_7',
  'concepto_del_metodo_7',
  'metodo_valuacion_8',
  'concepto_del_metodo_8',
  'metodo_valuacion_9',
  'concepto_del_metodo_9',
  'Longitud',
  'Latitud',
]

Por otro lado tenemos algunas que consideramos que pueden arrojan informacion pero necesitan trabajo adicional, mas que todo preprocesar intensamente la data, para extraerle un score o sentimiento:

[
  'descripcion_clase_inmueble',
  'perspectivas_de_valorizacion',
  'actualidad_edificadora',
  'comportamiento_oferta_demanda',
  'observaciones_generales_inmueble',
  'observaciones_estructura',
  'observaciones_generales_construccion',
  'observaciones_dependencias',
]

Hay otros que pueden agruparse, por ejemplo

## Influencia de la zona

[
  'departamento_inmueble',
  'municipio_inmueble',
  'barrio',
  'descripcion_general_sector',
  'direccion_inmueble_informe',
  'descripcion_general_sector',
]

Un caso interesante aca es el de 'sector' pues usualmente rural siempre vale menos que urbano por metro cuadrado. Sin embargo combinandolo con los anteriores puede dar mas informacion sobre la zona especifica, por ejemplo inidicar que un apartamento en la zona urbana tiene mayor valor que un lote urbano.

Pero las 3 ultimas son las que dan mas informacion, pues indican con palabras si el barrio o la ciudad estan cotizadas. Creemos que usar tambien departamento, municipio o barrio implicaria en la practica hacer una etiquetacion de tales columnas (es bogota bueno, malo o no afect al valor?). Por eso se incluyo al final a las 3 primeras para eliminar, pues no aportan tanta data como se quisiera.

## Influencia de la estructura

[
  'observaciones_generales_inmueble',
  'observaciones_estructura',
  'observaciones_dependencias',
  'observaciones_generales_construccion',
]

## Area, altura, dimensiones

[
  'observaciones_altura_permitida',
  'area_privada',
  'area_garaje',
  'area_deposito',
  'area_terreno',
  'area_construccion',
  'area_otros',
  'area_libre',
]

Aca podria pensarse que el area total y el area contruida serian los valores mas importantes, y otros valores no ayudan individualmente, asi mismo la atura no se tima en cuenta.

## Valor

Aca solo tomamos el valor total, las demas quedan borradas

First half here

Second half here